In [ ]:
import os
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

uploaded_path = r"C:\path\to\sales_task7.csv.xlsx"   
db_path = r"C:\path\to\sales_data.db"
chart_path = r"C:\path\to\sales_chart_task7.png"

df_raw = None
try:
    df_raw = pd.read_excel(uploaded_path)
    print("Read as Excel.")
except Exception:
    df_raw = pd.read_csv(uploaded_path)
    print("Read as CSV.")

if df_raw.shape[0] == 0:
    raise SystemExit("Input file is empty. Provide a file with at least one data row.")

product_col = quantity_col = price_col = None
for col in df_raw.columns:
    low = str(col).lower()
    if product_col is None and any(k in low for k in ("product","item","name","sku")):
        product_col = col
    if quantity_col is None and any(k in low for k in ("qty","quantity","count","units")):
        quantity_col = col
    if price_col is None and any(k in low for k in ("price","unit_price","amount","cost","rate")):
        price_col = col

cols = list(df_raw.columns)
if product_col is None or quantity_col is None or price_col is None:
    product_col, quantity_col, price_col = cols[0], cols[1], cols[2]
    print("Fallback mapping to first three columns:", product_col, quantity_col, price_col)

df = pd.DataFrame({
    "product": df_raw[product_col].astype(str),
    "quantity": pd.to_numeric(df_raw[quantity_col], errors='coerce').fillna(0).astype(int),
    "price": pd.to_numeric(df_raw[price_col], errors='coerce').fillna(0.0).astype(float),
    "order_date": df_raw[df_raw.columns[2]].astype(str) if len(df_raw.columns) > 3 else None
})

if os.path.exists(db_path):
    os.remove(db_path)
conn = sqlite3.connect(db_path)
cur = conn.cursor()
cur.execute("""
CREATE TABLE sales (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    product TEXT,
    quantity INTEGER,
    price REAL,
    order_date TEXT
)
""")
conn.commit()
df.to_sql("sales", conn, if_exists="append", index=False)

query = """
SELECT product,
       SUM(quantity) AS total_qty,
       SUM(quantity * price) AS revenue
FROM sales
GROUP BY product
ORDER BY revenue DESC;
"""
df_summary = pd.read_sql_query(query, conn)
print(df_summary)

totals = pd.read_sql_query("SELECT SUM(quantity) AS total_qty_all, SUM(quantity * price) AS total_revenue_all FROM sales", conn)
print(totals)

if not df_summary.empty:
    df_summary.plot(kind='bar', x='product', y='revenue', legend=False)
    plt.xlabel("Product")
    plt.ylabel("Revenue")
    plt.title("Revenue by Product")
    plt.tight_layout()
    plt.savefig(chart_path)
    plt.show()

conn.close()
